![](https://camo.githubusercontent.com/cdc7763be321bde0219f2d68ff51a8fc859adaaa/68747470733a2f2f7773312e73696e61696d672e636e2f6c617267652f303036744e6337396c7931666d70747132736e76396a33306a383038743734612e6a7067)

In [2]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10
import torch.nn.functional as F

In [3]:
def conv3x3(in_channel, out_channel, stride=1):
    return nn.Conv2d(in_channel, out_channel, 3, stride=stride, padding=1, bias=False)

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel, same_shape=True):
        super(ResidualBlock, self).__init__()
        self.same_shape = same_shape
        stride = 1 if self.same_shape else 2

        self.conv1 = conv3x3(in_channel, out_channel, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channel)

        self.conv2 = conv3x3(out_channel, out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        if not self.same_shape:
            self.conv3 = nn.Conv2d(in_channel, out_channel, 1, stride=stride)

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(self.bn1(out), True)
        out = self.conv2(out)
        out = F.relu(self.bn2(out), True)

        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(x + out, True)

In [8]:
# 输入输出形状相同
test_net = ResidualBlock(32, 32)
test_x = Variable(torch.zeros(1, 32, 96, 96))
print('input: {}'.format(test_x.shape))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

input: torch.Size([1, 32, 96, 96])
output: torch.Size([1, 32, 96, 96])


In [9]:
# 输入输出形状不同
test_net = ResidualBlock(3, 32, False)
test_x = Variable(torch.zeros(1, 3, 96, 96))
print('input: {}'.format(test_x.shape))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

input: torch.Size([1, 3, 96, 96])
output: torch.Size([1, 32, 48, 48])


In [28]:
class ResNet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(ResNet, self).__init__()
        self.verbose = verbose

        self.block1 = nn.Conv2d(in_channel, 64, 7, 2)
        self.block2 = nn.Sequential(
            nn.MaxPool2d(3, 2),
            ResidualBlock(64, 64),
            ResidualBlock(64, 64)
        )
        self.block3 = nn.Sequential(
            ResidualBlock(64, 128, False),
            ResidualBlock(128, 128)
        )
        self.block4 = nn.Sequential(
            ResidualBlock(128, 256, False),
            ResidualBlock(256, 256)
        )
        self.block5 = nn.Sequential(
            ResidualBlock(256, 512, False),
            ResidualBlock(512, 512),
            nn.AvgPool2d(3)
        )
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [29]:
test_net = ResNet(3, 10, True)
test_x = Variable(torch.zeros(1, 3, 96, 96))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

block 1 output: torch.Size([1, 64, 45, 45])
block 2 output: torch.Size([1, 64, 22, 22])
block 3 output: torch.Size([1, 128, 11, 11])
block 4 output: torch.Size([1, 256, 6, 6])
block 5 output: torch.Size([1, 512, 1, 1])
output: torch.Size([1, 10])


In [30]:
def data_tf(x):
    x = x.resize((96, 96), 2)  # 将图片放大到 96 x 96
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5  # 标准化，这个技巧之后会讲到
    x = x.transpose((2, 0, 1))  # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x


train_set = CIFAR10('./data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

In [31]:
net = ResNet(3, 10)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [32]:
from utils import train

train(net, train_data, test_data, 20, optimizer, criterion)

D:\Workspace\Samples\pytorch_lab\chapter03_cnn\utils.py:54: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
D:\Workspace\Samples\pytorch_lab\chapter03_cnn\utils.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 1.434181, Train Acc: 0.478061, Valid Loss: 1.687309, Valid Acc: 0.453125, Time 00:00:53
Epoch 1. Train Loss: 0.993083, Train Acc: 0.646200, Valid Loss: 1.634796, Valid Acc: 0.462322, Time 00:00:58
Epoch 2. Train Loss: 0.772935, Train Acc: 0.730479, Valid Loss: 0.979879, Valid Acc: 0.660799, Time 00:00:58
Epoch 3. Train Loss: 0.613819, Train Acc: 0.786845, Valid Loss: 1.261685, Valid Acc: 0.599684, Time 00:00:58
Epoch 4. Train Loss: 0.478512, Train Acc: 0.836857, Valid Loss: 1.121621, Valid Acc: 0.646559, Time 00:00:58
Epoch 5. Train Loss: 0.364775, Train Acc: 0.874680, Valid Loss: 0.973697, Valid Acc: 0.701839, Time 00:00:59
Epoch 6. Train Loss: 0.277485, Train Acc: 0.905591, Valid Loss: 1.949605, Valid Acc: 0.549545, Time 00:00:58
Epoch 7. Train Loss: 0.193986, Train Acc: 0.935362, Valid Loss: 1.891139, Valid Acc: 0.560819, Time 00:00:59
Epoch 8. Train Loss: 0.142970, Train Acc: 0.952546, Valid Loss: 1.282867, Valid Acc: 0.665150, Time 00:00:58
Epoch 9. Train Loss